In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from biolearn.data_library import DataLibrary

In [2]:
man=pd.read_table('../Blood-DNA-Methylation-Atlas/ref/HM450.hg38.manifest.tsv',index_col='Probe_ID')[['CpG_chrm','CpG_beg']]
man=man.dropna().copy()
man['CpG_beg']=man['CpG_beg'].astype(int)
man.columns=[0,1]
man['cg']=man.index

In [3]:
%%time 
wb_='data/cell/GSE41169_series_matrix.txt'
age=pd.read_table(wb_,skiprows=32+14,nrows=0,index_col=0).columns.str.strip('age: ').astype(float).tolist()
wb=pd.read_table(wb_,skiprows=32+14+26,index_col=0).drop('!series_matrix_table_end').round(3)

CPU times: user 7.6 s, sys: 936 ms, total: 8.53 s
Wall time: 8.61 s


In [4]:
%%time 
wb_t=wb.T
wb_t['age']=age
wb_cor=wb_t.corrwith(wb_t['age']).drop('age').sort_values()

CPU times: user 2min 2s, sys: 147 ms, total: 2min 2s
Wall time: 2min 2s


In [9]:
hmr=pd.read_table('/labs/mpsnyder/moqri/epi_deconv/data/arc/hmr/neo',header=None,index_col=3,usecols=[0,1,2,3])
hmr['b']=hmr[0].str[3:].astype(int)*10**9+hmr[1]
hmr['e']=hmr[0].str[3:].astype(int)*10**9+hmr[2]
hmr=hmr.sort_values('b')

In [8]:
man=man.merge(wb_cor.rename('cor'),left_index=True,right_index=True)
man=man[man[0].isin('chr'+str(c) for c in range(1,23))].copy()
man['pos']=man[0].str[3:].astype(int)*10**9+man[1]
man=man.sort_values('pos')

In [21]:
cur>=len(hmr)

True

In [22]:
%%time
l=[]
cur=0
for i in range(len(man)):
    if cur>=len(hmr):
        l.append('nan')
    else: 
        pos=man['pos'].iloc[i]
        cg=man['cg'].iloc[i]
        b=hmr['b'].iloc[cur]
        e=hmr['e'].iloc[cur]        
        if pos<b:
            l.append('nan')
        if pos>=b and pos<=e:
            l.append(cur)
        if pos>e:
            l.append('nan')
            cur+=1

CPU times: user 22.4 s, sys: 15.2 ms, total: 22.4 s
Wall time: 22.4 s


In [24]:
man['l']=l

In [25]:
mg=pd.DataFrame(man.groupby('l')['cor'].mean())
mg['c']=man.groupby('l')['cor'].count()
mg[mg['c']>=5].sort_values('cor').tail(10)

,cor,c
l,,
36634,0.449117,6
19194,0.449230,5
41521,0.458695,6
42161,0.459930,12
42406,0.475737,11
12403,0.477521,11
41187,0.481751,7
1989,0.533822,6
2189,0.581576,5


In [26]:
hmr.iloc[mg[mg['c']>=5].sort_values('cor').tail(10).index]

,0,1,2,b,e
3,,,,,
HYPO18969,chr17,28985870,28986481,17028985870,17028986481
HYPO43087,chr8,56447897,56448292,8056447897,8056448292
HYPO23856,chr19,41996936,41999519,19041996936,19041999519
HYPO24496,chr19,51887336,51888227,19051887336,19051888227
HYPO24741,chr19,57708673,57709644,19057708673,19057709644
HYPO36296,chr5,93620363,93621878,5093620363,5093621878
HYPO23522,chr19,35755461,35756232,19035755461,19035756232
HYPO1989,chr1,52842501,52844181,1052842501,1052844181
HYPO2189,chr1,68046821,68047168,1068046821,1068047168


In [46]:
%%time
hyp=[]
for i in top.index:
    if i in man.index:      
        ch=man.loc[i][0]
        pos=man.loc[i][1]
        if len(hmr[(hmr[0]==ch)&(hmr[1]<pos)&(hmr[2]>pos)])>0:
            hyp.append(hmr[(hmr[0]==ch)&(hmr[1]<pos)&(hmr[2]>pos)].index.tolist()[0])
        else:
            hyp.append('nan')
    else:
        hyp.append('nan')

KeyboardInterrupt: 

In [47]:
top['hyp']=hyp

ValueError: Length of values (367925) does not match length of index (485577)

In [ ]:
thyp=top.groupby('hyp')['hyp'].mean().sort_values().tail(50)

In [ ]:
hmr.merge(thyp,left_index=True,right_index=True).sort_values('hyp').tail(10).head(10)

In [ ]:
%%time
mono_='data/cell/GSE56046_methylome_normalized.txt'
mono=pd.read_table(mono_,index_col=0,usecols=[0]+list(range(1,201,2)))
p2=pow(2,mono).round(3)
mono=p2/(p2+1)
age=pd.read_table('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE56nnn/GSE56046/matrix/GSE56046_series_matrix.txt.gz',skiprows=46,
             index_col=0).iloc[10].str.strip('age (yrs): ').astype(int)
mono_t=mono.T
mono_t['age']=age.head(100).values
mono_cor=mono_t.corrwith(mono_t['age']).drop('age').sort_values()

In [ ]:
%%time
t_='data/cell/GSE56581_methylome_normalized.txt'
t=pd.read_table(t_,index_col=0,usecols=[0]+list(range(1,201,2)))
p2=pow(2,t).round(3)
t=p2/(p2+1)
age=pd.read_table('https://ftp.ncbi.nlm.nih.gov/geo/series/GSE56nnn/GSE56581/matrix/GSE56581_series_matrix.txt.gz',skiprows=40,
             index_col=0).iloc[10].str.strip('age (yrs): ').astype(int)
t_t=t.T
t_t['age']=age.head(100).values
t_cor=t_t.corrwith(t_t['age']).drop('age').sort_values()

In [ ]:
%%time
liver_='data/cell/GSE48325_series_matrix.txt'
liver=pd.read_table(liver_,skiprows=27+33,index_col=0).round(3)
age=pd.read_table(liver_,skiprows=27,nrows=50,index_col=0).iloc[8].str.strip('age: ').astype(int)
liver_t=liver.T
liver_t['age']=age.values
liver_cor=liver_t.corrwith(liver_t['age']).drop('age').sort_values()

In [ ]:
dd=pd.concat([wb_cor,t_cor,mono_cor,liver_cor],axis=1).drop('!series_matrix_table_end')
dd.corr()

In [ ]:
t_hm=pd.read_table('../Blood-DNA-Methylation-Atlas/data/merge/hm450/cd4.meth',header=None,skiprows=860,index_col=3)[[0,1,4]]
m_hm=pd.read_table('../Blood-DNA-Methylation-Atlas/data/merge/hm450/mono.meth',header=None,skiprows=860,index_col=3)[[0,1,4]]